< [Data Preprocessing and Visualization](../ica04/Data_Preprocessing_and_Visualization.ipynb) | Contents (TODO) |  [Distance and Similarity](../ica06/Distance_and_Similarity.ipynb) >

<a href="https://colab.research.google.com/github/stephenbaek/bigdata/blob/master/in-class-assignments/ica05/Supervised_Learning.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

# Introduction to Supervised Learning for Big Data

In this example, we will take a look at the issues regarding supervised learning in the context of big data. Especially, the computational speed is the major concern we will address here.

Before we begin, here's a simple trick you can use to measure the time elapsed for an operation.

In [ ]:
import time

In [ ]:
start_time = time.time()
time.sleep(3)  # an operation you want to evaluate
elapsed_time = time.time() - start_time
print('Elapsed time: {} seconds'.format(elapsed_time))

With this simple trick in hands, let's measure how long it takes to solve a linear system.

First, let us consider matrices $X\in \mathbb{R}^{N\times d}$ and $Y \in \mathbb{R}^{N\times 1}$ for some positive integers $N$ and $d < N$.

In [ ]:
import numpy as np

In [ ]:
N = 10000
d = 500
X = np.random.normal(loc=0, scale=1, size=[N, d])
Y = np.random.normal(loc=0, scale=1, size=[N, 1])

print(X)
print(Y)

For a linear system of equations $Y = XA$, the least square solution to this system is known as:

\begin{equation*}
A = ((X^\top X)^{-1}X^\top)Y
\end{equation*}

To compute this, a straightforward approach would be to (1) compute $X^\top X$ first, (2) take the inverse $(X^\top X)^{-1}$, (3) multiply $X^\top$ to the result, and finally (4) multiply $Y$. The following is an analysis of how much of computational time is requred for each of the steps.

In [ ]:
start_time = time.time()
XTX = np.matmul(X.T, X)
XTX_elapsed_time = time.time() - start_time
print('Elapsed time for XTX: {} seconds'.format(XTX_elapsed_time))

start_time = time.time()
inv = np.linalg.inv(XTX)
inv_elapsed_time = time.time() - start_time
print('Elapsed time for the inverse: {} seconds'.format(inv_elapsed_time))

start_time = time.time()
invXT = np.matmul(inv, X.T)
invXT_elapsed_time = time.time() - start_time
print('Elapsed time for the inverse times XT: {} seconds'.format(invXT_elapsed_time))

start_time = time.time()
A = np.matmul(invXT, Y)
A_elapsed_time = time.time() - start_time
print('Elapsed time for the inverse times XT times Y: {} seconds'.format(A_elapsed_time))

print('Total: {} seconds'.format(XTX_elapsed_time + inv_elapsed_time + invXT_elapsed_time + A_elapsed_time))

Now, a simple trick can make a huge difference in computational time. Consider the same equation as above, but this time, let us switch the order of computation a little bit.

\begin{equation*}
A = (X^\top X)^{-1}(X^\top Y)
\end{equation*}

That is, this time, we are going to (1) compute $X^\top X$ first, (2) take the inverse $(X^\top X)^{-1}$, (3) compute $X^\top Y$, and finally (4) multiply $(X^\top X)^{-1}$ and $X^\top Y$. Steps (1) and (2) are the same, but (3) and (4) is in different order. Let's take a look at how much time is required to compute the solution with this strategy.

In [ ]:
start_time = time.time()
XTX = np.matmul(X.T, X)
XTX_elapsed_time = time.time() - start_time
print('Elapsed time for XTX: {} seconds'.format(XTX_elapsed_time))

start_time = time.time()
inv = np.linalg.inv(XTX)
inv_elapsed_time = time.time() - start_time
print('Elapsed time for the inverse: {} seconds'.format(inv_elapsed_time))

start_time = time.time()
XTY = np.matmul(X.T, Y)
XTY_elapsed_time = time.time() - start_time
print('Elapsed time for XTY: {} seconds'.format(XTY_elapsed_time))

start_time = time.time()
A = np.matmul(inv, XTY)
A_elapsed_time = time.time() - start_time
print('Elapsed time for the inverse times XTY: {} seconds'.format(A_elapsed_time))

print('Total: {} seconds'.format(XTX_elapsed_time + inv_elapsed_time + XTY_elapsed_time + A_elapsed_time))

Notice the significant reduction of computation time?

### Assignment
- Which step shows the greatest difference?
- Why?
- Fix $d = 500$ but try to increase $N$ from 10,000 to 20,000, 50,000, and 100,000. How does the computation time chanbge? Is there any trend?
- Fix $N = 10000$ but increase $d$ from 500 to 1,000, 2,000, and 5,000. How does the computation time change? Is there any trend?

### Note: Advanced Profiling

Measuring times for running operations part by part is called profiling. Using `time` library is quite simple, but sometimes we may need some more advanced method. For example, you may have already noticed that the computation time of the same code can vary each time you run the code.

One way of profiling your code is by using `%timeit` tag in front of the line you want to evaluate. For example:
```python
%timeit inv = np.linalg.inv(XTX)
```
runs `inv = np.linalg.inv(XTX)` multiple times and take the average and standard deviation of the computation time.

Another way of doing it is by using `%prun` tag in front of the line. For instance:
```python
%prun inv = np.linalg.inv(XTX)
```
will provide more in-depth breakdown of the process. If you are, however, not so familiar with computer programing, `%prun` might be too much, as it gives too detailed information. In this case, you should just be fine with `%timeit` or the `time.time()` method.

In [ ]:
%timeit inv = np.linalg.inv(XTX)

In [ ]:
%prun inv = np.linalg.inv(XTX)

## Distributed Optimization

Despite the computational trick above, it may still be difficult to compute the pseudo-inverse ($(X^\top X)^{-1}X^\top Y$) due to limitations in the memory space, etc. In fact, many big data analytics frameworks (including Spark) use distributed optimization strategies. Below, we are going to see an example of distributed gradient descent algorithm to get a feel of how things work.

[Gradient descent](https://en.wikipedia.org/wiki/Gradient_descent) is a popular first order optimization algorithm for finding the local minimum. We are not going to delve into what gradient descent algorithm really is, but here is a iterative update formula used by gradient descent algorithm:

$\theta \longleftarrow \theta - \alpha \frac{\partial\mathcal{L}}{\partial \theta}$

where $\theta$ is model parameters ($w$ and $b$ in case of linear regression). $\frac{\partial\mathcal{L}}{\partial \theta}$ is the first order derivative (gradient) of the loss (error) function $\mathcal{L}$ with respect to the model parameter $\theta$.

Recall, the loss function is formulated as:

$\mathcal{L}(w, b) = \frac{1}{N}\sum_{i=1}^N\|x^{(i)}w+b - y^{(i)}\|^2$

If we expand the summation symbol, the loss function will look like this:

$\mathcal{L} = \mathcal{L}^{(1)} + \mathcal{L}^{(2)} + \cdots + \mathcal{L}^{(N)}$

Note $\mathcal{L}^{(i)}$ indicates the $i$-th term in the loss function, corresponding to the $i$-th data point.
When we omit the superscripts for simplicity, each term $\mathcal{L}^{(i)}$ should look like this:

$\mathcal{L}^{(i)}=\frac{1}{N}\|xw+b - y\|^2=\frac{1}{N}\|\hat{y} - y\|^2$

Here a new notation $\hat{y}:= xw+b$ has been introduced to indicate the output predicted by the linear regression model. Under this notation, it takes only simple calculus to compute the first order derivatives:

$\frac{\partial\mathcal{L}^{(i)}}{\partial w} = \frac{2}{N}(\hat{y} - y) x^\top$

$\frac{\partial\mathcal{L}^{(i)}}{\partial b} = \frac{2}{N}(\hat{y} - y)$

Recall that the above derivatives are for the $i$-th term in the loss function, associated only with the $i$-th data point. In other words, the global gradient (the gradient of the entire loss function) is simply the summation of the local gradients (the gradient of $i$-th term), which can be computed independently of the other data points:

$\frac{\partial\mathcal{L}}{\partial W} = \frac{\partial\mathcal{L}^{(1)}}{\partial W} + \frac{\partial\mathcal{L}^{(2)}}{\partial W} + \cdots + \frac{\partial\mathcal{L}^{(N)}}{\partial W}$

This means that, no mater how data is distributed, we can simply compute the $i$-th local gradient at each data point $(i)$ and later on aggregate them to produce the global gradient.

To understand the concept more clearly, here's an example of 10,000 data points splitted into two (simulated) computing nodes. First, let us create a random data set with $N=10,000$ and $d=10%.

In [ ]:
N = 10000
d = 10
X = np.random.normal(loc=0, scale=1, size=[N, d])
Y = np.random.normal(loc=0, scale=1, size=[N, 1])

Now, here's how we split the data set.

In [ ]:
N_node1 = N//2
N_node2 = N - N_node1

X_node1 = X[:N_node1, :]
Y_node1 = Y[:N_node1]
X_node2 = X[N_node1:, :]
Y_node2 = Y[N_node1:]

Although, we do not physically split the data set, we pretend `X_node1` and `Y_node1` are only accessible from `node1` and `X_node2` and `Y_node2` are only accessible from `node2`. Now, at a `master` node, the following routine will be executed:

In [ ]:
# Initialize model parameters
w = np.random.uniform(-1.0, 1.0, [d,1])
b = np.random.uniform(-1.0, 1.0)

MAX_ITER = 1000
learning_rate = 0.01
for i in range(MAX_ITER):
    # Talk to node1 and ask it to compute the local gradient.
    # (Pretend the following three lines are computed on node1)
    err_node1 = np.matmul(X_node1, w) - Y_node1
    dldw_node1 = np.mean(2*err_node1*X_node1, axis=0, keepdims=True)
    dldb_node1 = np.mean(2*err_node1, axis=0, keepdims=True)

    # Simultaneously, talk to node2 and ask the same.
    # (Pretend the following three lines are computed on node2)
    err_node2 = np.matmul(X_node2, w) - Y_node2
    dldw_node2 = np.mean(2*err_node2*X_node2, axis=0, keepdims=True)
    dldb_node2 = np.mean(2*err_node2, axis=0, keepdims=True)

    # Aggregate the gradients by weighting them with the number of data available at each node.
    dldw = (N_node1/N)*dldw_node1 + (N_node2/N)*dldw_node2
    dldb = (N_node1/N)*dldb_node1 + (N_node2/N)*dldb_node2

    # Update the model with the global gradient.
    w -= learning_rate*dldw.T
    b -= learning_rate*dldb.T

    # If the solution does not improve much, break out of the for loop
    if np.mean(dldw) < 1e-06:
        break

Of course, the above is quite simplified implementation of distributed gradient descent. However, with such a simple version, we can see the result is almost the same as the pseudo-inverse solution.

In [ ]:
XTX = np.linalg.inv(np.matmul(X.T, X))
XTY = np.matmul(X.T, Y)
w_true = np.matmul(XTX, XTY)
print(w)   # distributed gradient descent solution
print(w_true)  # pseudo-inverse solution (ground truth)

With the above example, I hope the concept is clear now in your mind. Even if you are still not so confident if you could actually right a code like the above from the scratch, you should be completely fine, as far as you have a clear big picture. In fact, the implementation of the distributed optimization and such are taken cared by Spark. Instead, as a data scientist, you just need to have a basic understanding of how it actually works under the hood. So don't worry.

## Linear Regression in Spark

Spark has all the powerful, well-optimized implementation of such distributed optimization methods (and many more) under the hood. In fact, from the user's perspective, 99% of the times, you don't really need to care about what's happening behind the scene. Spark will pick the most suitable optimization algorithm for you and do all the heavy-lifting behind the scene.

To see how it works, let's first configure Spark on Colab. (If you are running this notebook on your local machine and have configured Spark already, you can skip this cell.)

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

import findspark
findspark.init()

Now, let's download a dataset to play with. For this tutorial, we will use the [wine quality dataset](https://archive.ics.uci.edu/ml/datasets/Wine+Quality) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php).

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

In the previous lecture, we learned how to read a CSV file as a Spark DataFrame. Here we repeat what we've learned:

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('wine').getOrCreate()

In [ ]:
df = spark.read.csv('winequality-red.csv', header=True, sep=';', inferSchema=True)
df.show()

Note that `sep` argument is set to `';'`. This is because the wine quality dataset is written in a weird (?) convention where the values are separated by semicolon (;) instead of comma (,).

Now, in order to use Spark ML library, you must first convert columns into a feature vector. For this data set, we are supposed to predict the quality grade of a wine (last column) using features such as acidity, sulfur dioxide contents, pH, density, etc. (all the other columns).

To this end, Spark provides a handy method called `VectorAssembler` to produce a feature vector by assembling DataFrame columns.

In [ ]:
from pyspark.ml.feature import VectorAssembler
# Creates a new column called 'features' that contains feature vectors.
assembler = VectorAssembler(inputCols=df.columns[:-1], outputCol="features")
df_vec = assembler.transform(df)
df_vec.show()

Now that we created feature vectors, let us split the dataset into two---a training set and a test set. Spark DataFrame offers you an off-the-shelf method to do so:

In [ ]:
df_train, df_test = df_vec.randomSplit([0.7, 0.3])  # 70% of the original data will be used for training and 30% for testing

Now, the actual learning part is quite simple as described below.

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol='features', labelCol='quality') 
lr_model = lr.fit(df_train)

After the training is done, results can be found by calling members of the model such as `lr_model.coefficients` or `lr_model.intercept`. For more details, see https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegressionModel

In [ ]:
print( lr_model.coefficients )  # slope of the linear equation
print( lr_model.intercept )     # intercept of the line

In addition, you can also view the summary of training by calling members of the model summary (`lr_model.summary`). Again, I'll leave the full list of functions to [this URL](https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.regression.LinearRegressionModel), but only show some selected examples instead.

In [ ]:
lr_model.summary.residuals

In [ ]:
lr_model.summary.rootMeanSquaredError
# lr_model.summary.meanAbsoluteError
# lr_model.summary.meanSquaredError

In [ ]:
lr_model.summary.coefficientStandardErrors

In [ ]:
lr_model.summary.r2
lr_model.summary.r2adj
# lr_model.summary.pValues

A model trained using Spark can be tested by using `evaluate()` method.

In [ ]:
evaluation_summary = lr_model.evaluate(df_test)

Evaluation summary object is essentially the same as the model summary object. In other words, what you've seen above are applicable to the evaluation summary:

In [ ]:
evaluation_summary.rootMeanSquaredError

In [ ]:
lr_model.summary.totalIterations

## Transformers and Estimators

I hope you now have a better feel of how things work in Spark ML. Well, in the above exercise, I intentionally skipped many details for the sake of making a gentle introduction. However, there are a few essential concepts that you need to master, in order to truely take advantage of Spark ML. For this, let's make the problem a slightly more complicated by appending a white wine data set (in addition to red). The additional white wine data set can be downloaded from the following link.

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv

Now, let's read them and join them to produce a larger data set. While doing so, we are going to add an additional column called `type` to indicate if it is a red wine or a white wine. After the DataFrame exercise we have had, the following code should be quite straightforward. (If not, try to go back and brush yourself up on this.)

In [ ]:
df_red = spark.read.csv('winequality-red.csv', header=True, sep=';', inferSchema=True)
df_white = spark.read.csv('winequality-white.csv', header=True, sep=';', inferSchema=True)

from pyspark.sql.functions import lit    # to add a column filled with a constant value
df_red = df_red.withColumn('type', lit('red'))
df_white = df_white.withColumn('type', lit('white'))

df_red.show()
df_white.show()

In [ ]:
df = df_red.union(df_white)
print(df.count())      # note the number of rows of the union = rows in red + rows in white. 
print(df_red.count())
print(df_white.count())

Now, here's the main topic of the day. What we've just created is a DataFrame containing a column with a categorical variable. The category values are in string, so we need to somehow convert this into numerical values. This is a quite common, day-to-day task for data scientists. (Yes, of course, we could've created `type` column with numerical indices at first place, but we are simulating a real-world situation here.)

In spark, there are some helper methods to do such, called `StringIndexer` and `OneHotEncoderEstimator`. StringIndexer literally is a helper method that converts string type categorical variables to numerical indices. OneHotEncoderEstimator, on the other hand, is another helper method that converts a numerical index into a [one-hot encoding](https://en.wikipedia.org/wiki/One-hot). Those are quite frequently used features, as you can imagine.

In [ ]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol = 'type', outputCol = 'type_index')
df = indexer.fit(df).transform(df)
df.show()

In [ ]:
from pyspark.ml.feature import OneHotEncoderEstimator
encoder = OneHotEncoderEstimator(inputCols=['type_index'], outputCols=['type_dummy'])
df = encoder.fit(df).transform(df)
df.show()

Now, in the above, we used `fit()` and `transform()` methods on the DataFrame `df`. These two functions are, in fact, related two of the core concepts we aim to learn in this session---namely **transformer** and **estimator**.

According to the [Spark Official Documentation](https://spark.apache.org/docs/latest/ml-pipeline.html), a **transformer** is an abstraction that includes feature transformers and learned models. Simply speaking, transformers are an object equipped with logics that can convert one DataFrame into another. For example, a transformer may take a DataFrame, read a column (e.g. categorical variables), map it into a new column (e.g. numerical indices), and output a new DataFrame with the mapped column appended at the end. For another example, a transformer may also take a DataFrame, read feature vectors in a column, predict the label for each feature vector, and output a new DataFrame with predicted labels. Therefore, a (trained) machine learning model is a transformer, in a sense that in transforms an input DataFrame into an output DataFrame.

On the other hand, an **estimator** abstracts the concept of a learning algorithm or any algorithm that fits or trains on data. Estimators literally takes a DataFrame and *estimates* parameters by using the values in the DataFrame. For example, `LinearRegression` we saw above is an estimator and it calls `fit()` to train a `LinearRegressionModel`, which is an output of `fit()`. The output `LinearRegressionModel` on the other hand is a transformer, which can be used to transform a test DataFrame to produce predicted labels.

An easy way to distinguish transformers and estimators is by looking at the methods they have. Technically, a transformer implements a method `transform()`, which converts one DataFrame into another, generally by appending one or more columns. An estimator, on the other hand, implements a method `fit()`, which takes in a DataFrame and produces, typically a transformer.

With all this, let's decode the cells above. First of all, `StringIndexer` is an estimator which takes in a DataFrame and produces a transformer. How do I know that? Well, it comes with `fit()` function after all, so I can safely assume that it is an estimator. More specifically, the StringIndexer estimator estimates the number of different categories from a DataFrame and generates a transformer that implements a logic to map each different category into a numerical value. How many different categories are in a DataFrame varies problem by problem, so you need to estimate that parameter (number of categories) first, before to be able to transform a DataFrame.

The output of `StringIndexer.fit()` is, as said above, a transformer. It contains a logic to map each string category into a numerical index. Calling `transform()` will apply such a logic and will transform an input DataFrame into an output DataFrame.

Does the syntax `indexer.fit(df).transform(df)` make sense now? You can interpret the `OneHotEncoderEstimator` codes similarly. Knowing that you all are smart people, I'm not going to explicitly explain what they are line by line.


Great, now you just mastered one of the most important notions in Spark. On a minor note, you will notice that the one hot encoded representation in `type_dummy` column is kind of weird:

In [ ]:
df.show()

As you can see above, values in the `type_dummy` column looks like `(1,[],[])`, and you may wonder, what the...? Well, this is in fact a sparse vector representation used by Spark. If you think about it, one hot encoding is quite inefficient way of representing a class. In our case, we have only two classes, namely `red` and `white`. But in many cases, you will frequently see hundreds or thousands categories. In those cases, an one-hot encoding will end up looking like a thousand dimensional vector (a vector with a thousand entries) all filled with '0' except for one entry. So, it is a huge waste of memory to store unnecessary zeros in a DataFrame, especially if you are talking about a big data problem. In this reason, Spark uses a sparse representation in which, instead of storing all the entries with zeros, it only stores non-zero elements. To do so, you need three things. First you need to know the dimension of the vector, because you will skip many of the entries. Second, you need to know the positions of non-zero values, for an obvious reason. Lastly, you need to know what precisely those values are.

These three are, in fact, exactly what Spark stores to represent a one-hot encoding vector. The first number in the tuple `(1,[],[])` indicates the dimension of the one-hot encoding vector, which in this case is 1 (= number of categories - 1). The second object in the tuple is a list of positions of non-zero elements, which in this case is an empty array---meaning there is no non-zero element. Finally, the third object is a list of the actual values of the non-zero elements, which again, in our case is an empty array because we have no non-zero element. So, the reconstruction of the sparse representation `(1,[],[])` is going to be `[0]` in the dense representation---an one-dimensional vector with an element '0'.

Remember, the first rows contained a bunch of `red` wines. If you display rows for `white` wines, you will see some different one-hot encodings:

In [ ]:
df.where(df['type']=='white').show()

Did you notice now we have `(1,[0],[1.0])` instead of `(1,[],[])`? Again, the same interpretation: we have an one dimensional vector (`1`); the position of the non-zero element is at the 0-th position (`[0]`); the value of the non-zero element is 1.0 (`[1.0]`).

Okay, moving on. As a next step, we need to vectorize columns in order to train a linear regression model. Actually, we've done this above already, so we can pretty much just recycle the previous code. Just be careful that we ended up adding a few more columns as a result of StringIndexer and OneHotEncoderEstimator.

In [ ]:
assembler = VectorAssembler(inputCols=df.columns[:-4] + ['type_dummy'], outputCol="features")
df_vec = assembler.transform(df).select(['features', 'quality'])
df_vec.show()

By the way, did you realize that `VectorAssembler` was a transformer? Good. I bet you did. Did you also realize that while `StringIndexer` and `OneHotEncoderEstimator` were estimators, `VectorAssembler` was a straight up transformer? Can you guess why this is? Explain!

Finally, we can train a linear regression model with the additional `type_dummy` column as a regressor. 

In [ ]:
df_train, df_test = df_vec.randomSplit([0.7, 0.3])

lr = LinearRegression(featuresCol='features', labelCol='quality')
lr_model = lr.fit(df_train)

predicted = lr_model.evaluate(df_test)

Again, did you notice that `LinearRegression` has `fit()` method and thus is an estimator? And the `LinearRegressionModel` object, which in our case stored in `lr_model` variable, is a transformer?

So, as you can see, transformers and estimators are the core concepts in Spark. In fact, many of the examples we will see later in this course are going to be essentially just different estimators and transformers.


## Pipeline
One last thing: like what we saw in the DataFrame class, Spark builds upon the concept of 'lazy execution', meaning, until the user asks for actual numerical values, no number crunching is done. Instead, spark just keeps constructing a computational graph, illustrating the logic of how numbers are going to be computed. At the end, only when the user asks for an actual numerical result by calling, say `DataFrame.show()`, Spark actually flows the numbers through the graph and does actual number crunching. We saw this was because of an efficiency concern. The same principle is applied to transformers and estimators. In other words, estimators and transformers merely just construct a computational graph. Only when the user calls e.g. `DataFrame.show()`, the computational graph is actually executed and the numbers flow through the graph.

In some sense, the computational graph is like a 'pipeline', where data flows. As an architect, you add different shaped pipes to the pipeline. Only when you are ready to flow water, you unlock the valve and let water flow. Using the same analogy, in Spark, you can (and should) define a pipeline. More specifically, a Spark Pipeline is an object entailing how transformers and estimators (=pipes) are connected. When the user seeks for a numerical result, Spark runs numbers in a DataFrame (=water) through the Pipeline and produces a result.

To construct a Pipeline, we must define stages:

In [ ]:
stages = []  # create an empty list

indexer = StringIndexer(inputCol = 'type', outputCol = 'type_index')
encoder = OneHotEncoderEstimator(inputCols=['type_index'], outputCols=['type_dummy'])
assembler = VectorAssembler(inputCols=df.columns[:-4] + ['type_dummy'], outputCol="features")

stages += [indexer, encoder, assembler]  # append the estimators and transformers (ordering matters)

Now, setting up a pipeline is actually just a few lines of code:

In [ ]:
df = df_red.union(df_white)  # overwrite df by recreating it.

from pyspark.ml import Pipeline
pipeline = Pipeline(stages = stages)
pipelineModel = pipeline.fit(df)
df_vec = pipelineModel.transform(df)
df_vec.show()

Again, notice that a Pipeline itself is an estimator, which can `fit()` to a DataFrame to produce a transformer. The result of the transformer is a new DataFrame converted from the input DataFrame by applying the sequence of `indexer`--`encoder`--`assembler`. The output of the pipeline should be exactly the same as the one from the previous section.

If the output is the same, why do we bother to use a Pipeline? Well, it is first of all easier to code, as you might have noticed already. In stead of defining a bunch of new DataFrames after each transformer, we could just append all the estimators and transformers, put them in a single unified Pipeline estimator, and run a DataFrame at once. It also has an effect of optimizing the computational graph even further, as it becomes more explcit how the transformers and estimators should be connected.

For more information, you are strongly encouraged to study the official documentation of Spark: https://spark.apache.org/docs/latest/ml-pipeline.html

< [Data Preprocessing and Visualization](../ica04/Data_Preprocessing_and_Visualization.ipynb) | Contents (TODO) |  [Distance and Similarity](../ica06/Distance_and_Similarity.ipynb) >

<a href="https://colab.research.google.com/github/stephenbaek/bigdata/blob/master/in-class-assignments/ica05/Supervised_Learning.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>